In [120]:
import pandas as pd
from prophet import Prophet

In [121]:
df = pd.read_csv('data/m_and_e_stacked.csv')
df.head()

,Unnamed: 0,MSA/Submarket,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31
0,0,Dallas/Fort Worth Overall_market,1.80,1.79,1.78,1.77,1.77,1.76,1.76,1.76,1.76,1.77,1.77,1.77
1,1,Bachman Lake/Webb Chapel_market,1.63,1.65,1.66,1.67,1.66,1.69,1.68,1.68,1.68,1.69,1.69,1.69
2,2,Carrollton North_market,1.84,1.84,1.83,1.83,1.82,1.83,1.83,1.83,1.82,1.82,1.82,1.81
3,3,Carrollton South_market,1.84,1.84,1.83,1.83,1.82,1.83,1.84,1.82,1.84,1.83,1.83,1.84
4,4,Central Arlington_market,1.52,1.54,1.52,1.52,1.53,1.54,1.54,1.55,1.54,1.54,1.54,1.54


In [122]:
'''
new_dates = []
for i in df.columns:
    dates = i.split('-')
    if len(dates)==3:
        new_date = f'{dates[0]}-{dates[1]}-01'
    else:
        new_date = i
    new_dates.append(new_date)
        
df.columns = new_dates
df
'''

"\nnew_dates = []\nfor i in df.columns:\n    dates = i.split('-')\n    if len(dates)==3:\n        new_date = f'{dates[0]}-{dates[1]}-01'\n    else:\n        new_date = i\n    new_dates.append(new_date)\n        \ndf.columns = new_dates\ndf\n"

In [123]:
list_markets = list(set([i.split('_')[0] for i in df['MSA/Submarket']]))

In [124]:
market = 'Dallas/Fort Worth Overall'
df_filtered = df[df['MSA/Submarket'].str.contains(market)].drop('Unnamed: 0',axis=1)
df_filtered.index = df_filtered['MSA/Submarket']
transposed_df = df_filtered.T
transposed_df.drop('MSA/Submarket',axis=0)

MSA/Submarket,Dallas/Fort Worth Overall_market,Dallas/Fort Worth Overall_effective
2023-08-31,1.8,1.77
2023-09-30,1.79,1.77
2023-10-31,1.78,1.76
2023-11-30,1.77,1.74
2023-12-31,1.77,1.74
2024-01-31,1.76,1.73
2024-02-29,1.76,1.72
2024-03-31,1.76,1.72
2024-04-30,1.76,1.73
2024-05-31,1.77,1.73


In [125]:
transposed_df.columns = ['y','m']
transposed_df['ds'] = transposed_df.index
transposed_df.drop('MSA/Submarket',axis=0,inplace=True)
transposed_df['ds'] = pd.to_datetime(transposed_df['ds'])

In [126]:
m1 = Prophet()
m1.fit(transposed_df)

02:21:15 - cmdstanpy - INFO - Chain [1] start processing
02:21:15 - cmdstanpy - INFO - Chain [1] done processing


In [127]:
future = m1.make_future_dataframe(periods=5,freq='M')
forecast_df = m.predict(future)
forecast_df[['ds','yhat', 'yhat_lower', 'yhat_upper']].tail()

C:\Users\baker\Anaconda3\envs\realty-env\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


,ds,yhat,yhat_lower,yhat_upper
12,2024-08-31,1.774981,1.772023,1.778169
13,2024-09-30,1.777903,1.771654,1.784829
14,2024-10-31,1.780924,1.769319,1.794209
15,2024-11-30,1.783846,1.765750,1.805076
16,2024-12-31,1.786866,1.760975,1.815938


In [144]:
transposed_df.columns = ['m','y','ds']
transposed_df

,m,y,ds
2023-08-31,1.8,1.77,2023-08-31
2023-09-30,1.79,1.77,2023-09-30
2023-10-31,1.78,1.76,2023-10-31
2023-11-30,1.77,1.74,2023-11-30
2023-12-31,1.77,1.74,2023-12-31
2024-01-31,1.76,1.73,2024-01-31
2024-02-29,1.76,1.72,2024-02-29
2024-03-31,1.76,1.72,2024-03-31
2024-04-30,1.76,1.73,2024-04-30
2024-05-31,1.77,1.73,2024-05-31


In [145]:
m2 = Prophet()
m2.add_regressor('m')
m2.fit(transposed_df)

02:22:32 - cmdstanpy - INFO - Chain [1] start processing
02:22:32 - cmdstanpy - INFO - Chain [1] done processing


In [146]:
future = m2.make_future_dataframe(periods=5,freq='M')
future['m'] = forecast['yhat']

C:\Users\baker\Anaconda3\envs\realty-env\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [147]:
future_preds = m.predict(future)

In [148]:
future_preds[['ds','m','yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,m,yhat,yhat_lower,yhat_upper
12,2024-08-31,2.044674e-18,1.73,1.724068,1.736785
13,2024-09-30,2.553509e-18,1.73,1.709865,1.752792
14,2024-10-31,3.062344e-18,1.73,1.690781,1.776860
15,2024-11-30,3.571179e-18,1.73,1.663626,1.800774
16,2024-12-31,4.080015e-18,1.73,1.633166,1.828944


In [156]:
for i in list_markets:
    market = i
    df_filtered = df[df['MSA/Submarket'].str.contains(market)].drop('Unnamed: 0',axis=1)
    df_filtered.index = df_filtered['MSA/Submarket']
    transposed_df = df_filtered.T
    transposed_df.drop('MSA/Submarket',axis=0,inplace=True)
    transposed_df.columns = ['y','m']
    transposed_df['ds'] = transposed_df.index
    m1 = Prophet()
    m1.fit(transposed_df)
    future = m1.make_future_dataframe(periods=5,freq='M')
    forecast_df = m1.predict(future)
    print(forecast_df[['ds','yhat', 'yhat_lower', 'yhat_upper']].tail())
    transposed_df.columns = ['m','y','ds']
    m2 = Prophet()
    m2.add_regressor('m')
    m2.fit(transposed_df)
    future = m2.make_future_dataframe(periods=5,freq='M')
    future['m'] = forecast['yhat']
    future_preds = m.predict(future)
    print(future_preds[['ds','m','yhat', 'yhat_lower', 'yhat_upper']].tail())

02:33:27 - cmdstanpy - INFO - Chain [1] start processing
02:33:27 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\baker\Anaconda3\envs\realty-env\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
02:33:27 - cmdstanpy - INFO - Chain [1] start processing
02:33:27 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\baker\Anaconda3\envs\realty-env\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


           ds      yhat  yhat_lower  yhat_upper
12 2024-08-31  1.809294    1.805028    1.813554
13 2024-09-30  1.806015    1.794302    1.819031
14 2024-10-31  1.802626    1.779914    1.826995
15 2024-11-30  1.799347    1.763187    1.837145
16 2024-12-31  1.795958    1.743560    1.847487
           ds             m  yhat  yhat_lower  yhat_upper
12 2024-08-31  2.044674e-18  1.73    1.723223    1.737056
13 2024-09-30  2.553509e-18  1.73    1.707979    1.750806
14 2024-10-31  3.062344e-18  1.73    1.687418    1.770128
15 2024-11-30  3.571179e-18  1.73    1.661300    1.792781
16 2024-12-31  4.080015e-18  1.73    1.631945    1.817582


02:33:27 - cmdstanpy - INFO - Chain [1] start processing
02:33:27 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\baker\Anaconda3\envs\realty-env\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
02:33:28 - cmdstanpy - INFO - Chain [1] start processing


           ds      yhat  yhat_lower  yhat_upper
12 2024-08-31  1.782942    1.775712    1.789533
13 2024-09-30  1.783431    1.771753    1.794727
14 2024-10-31  1.783937    1.765454    1.802702
15 2024-11-30  1.784427    1.754802    1.812745
16 2024-12-31  1.784932    1.742492    1.824917


02:33:28 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\baker\Anaconda3\envs\realty-env\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
02:33:28 - cmdstanpy - INFO - Chain [1] start processing
02:33:28 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\baker\Anaconda3\envs\realty-env\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


...

02:33:47 - cmdstanpy - INFO - Chain [1] start processing
02:33:47 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\baker\Anaconda3\envs\realty-env\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
02:33:47 - cmdstanpy - INFO - Chain [1] start processing
02:33:47 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\baker\Anaconda3\envs\realty-env\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


           ds             m  yhat  yhat_lower  yhat_upper
12 2024-08-31  2.044674e-18  1.73    1.722972    1.737725
13 2024-09-30  2.553509e-18  1.73    1.708583    1.753587
14 2024-10-31  3.062344e-18  1.73    1.685483    1.773709
15 2024-11-30  3.571179e-18  1.73    1.659705    1.799565
16 2024-12-31  4.080015e-18  1.73    1.631265    1.825969
           ds      yhat  yhat_lower  yhat_upper
12 2024-08-31  1.504214    1.472493    1.535005
13 2024-09-30  1.498705    1.466725    1.529526
14 2024-10-31  1.493012    1.458000    1.525492
15 2024-11-30  1.487503    1.455160    1.520348
16 2024-12-31  1.481810    1.448118    1.515138


02:33:47 - cmdstanpy - INFO - Chain [1] start processing
02:33:47 - cmdstanpy - INFO - Chain [1] done processing


           ds             m  yhat  yhat_lower  yhat_upper
12 2024-08-31  2.044674e-18  1.73    1.723311    1.736279
13 2024-09-30  2.553509e-18  1.73    1.708107    1.751836
14 2024-10-31  3.062344e-18  1.73    1.685536    1.771593
15 2024-11-30  3.571179e-18  1.73    1.659767    1.796701
16 2024-12-31  4.080015e-18  1.73    1.629589    1.826745


C:\Users\baker\Anaconda3\envs\realty-env\lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
